# End-To-End Example with Tensorflow Keras and Hops - MNIST Classification
---
*DATA PREPARATION --> MODEL TRAINING --> MODEL SERVING --> MODEL MONITORING*

<font color='red'> <h3>Tested with TensorFlow 2.9.1</h3></font>

## Model Serving on [Hopsworks](https://github.com/logicalclocks/hopsworks)

![hops.png](../../images/hops.png)

## The `hops` python library

`hops` is a helper library for Hops that facilitates development by hiding the complexity of running applications and iteracting with services.

Have a feature request or encountered an issue? Please let us know on <a href="https://github.com/logicalclocks/hops-util-py">github</a>.

### Using the `experiment` module

To be able to run your Machine Learning code in Hopsworks, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function.

The `experiment` module provides an api to Python programs such as TensorFlow, Keras and PyTorch on a Hopsworks on any number of machines and GPUs.

An Experiment could be a single Python program, which we refer to as an **Experiment**. 

Grid search or genetic hyperparameter optimization such as differential evolution which runs several Experiments in parallel, which we refer to as **Parallel Experiment**. 

ParameterServerStrategy, CollectiveAllReduceStrategy and MultiworkerMirroredStrategy making multi-machine/multi-gpu training as simple as invoking a function for orchestration. This mode is referred to as **Distributed Training**.

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project's Experiments folder.

The directory could in practice be used to store other data that should be accessible after the experiment is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```

### Using the `hdfs` module
The `hdfs` module provides a method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`

```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

```python
# Downloading the mnist dataset to the current working directory
from hops import hdfs
mnist_hdfs_path = hdfs.project_path() + "Resources/mnist"
local_mnist_path = hdfs.copy_to_local(mnist_hdfs_path)
```

### Documentation
See the following links to learn more about running experiments in Hopsworks

- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/experiment.html">Learn more about experiments</a>
<br>
- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/hopsML.html">Building End-To-End pipelines</a>
<br>
- Give us a star, create an issue or a feature request on  <a href="https://github.com/logicalclocks/hopsworks">Hopsworks github</a>

### Managing experiments
Experiments service provides a unified view of all the experiments run using the `experiment` module.
<br>
As demonstrated in the gif it provides general information about the experiment and the resulting metric. Experiments can be visualized meanwhile or after training in a TensorBoard.
<br>
<br>
![Image7-Monitor.png](../../images/experiments.gif)

## Train a Mnist Classifier running a Experiment

In [1]:
MODEL_NAME = "mniste2e"
EXPERIMENT_NAME = "Mnist classifier for e2e example"

In [2]:
import os
import sys
import uuid
import random

import numpy as np

from tensorflow import keras
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as K

import math
from hops import tensorboard

import hsml
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from hops import hdfs

import pydoop.hdfs as pydoop

batch_size=32
num_classes = 10

# Provide path to train and validation datasets
train_filenames = [hdfs.project_path() + "TourData/mnist/train/train.tfrecords"]
validation_filenames = [hdfs.project_path() + "TourData/mnist/validation/validation.tfrecords"]

# Define input function
def data_input(filenames, batch_size=128, num_classes = 10, shuffle=False, repeat=None):

    def parser(serialized_example):
        """Parses a single tf.Example into image and label tensors."""
        features = tf.io.parse_single_example(
            serialized_example,
            features={
                'image_raw': tf.io.FixedLenFeature([], tf.string),
                'label': tf.io.FixedLenFeature([], tf.int64),
            })
        image = tf.io.decode_raw(features['image_raw'], tf.uint8)
        image.set_shape([28 * 28])

        # Normalize the values of the image from the range [0, 255] to [-0.5, 0.5]
        image = tf.cast(image, tf.float32) / 255 - 0.5
        label = tf.cast(features['label'], tf.int32)

        # Create a one hot array for your labels
        label = tf.one_hot(label, num_classes)

        return image, label

    # Import MNIST data
    dataset = tf.data.TFRecordDataset(filenames)

    # Map the parser over dataset, and batch results by up to batch_size
    dataset = dataset.map(parser)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=128)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.repeat(repeat)
    return dataset

# Define a Keras Model.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Compile the model.
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer= tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy']
             )

model.fit(data_input(train_filenames, batch_size), 
    verbose=0,
    epochs=3, 
    steps_per_epoch=5,
    validation_data=data_input(validation_filenames, batch_size),
    validation_steps=1
)

score = model.evaluate(data_input(validation_filenames, batch_size), steps=1)

# Export model
# WARNING(break-tutorial-inline-code): The following code snippet is
# in-lined in tutorials, please update tutorial documents accordingly
# whenever code changes.

export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
print('Saving trained model to: {}'.format(export_path))

tf.saved_model.save(model, export_path)

print('Done saving!')

metrics = {'accuracy': score[1]}

# export trained model

conn = hsml.connection()
mr = conn.get_model_registry()

input_example = np.random.randint(0, high=256, size=784, dtype=np.uint8)

input_schema = Schema(input_example)
output_schema = Schema([{'type': 'float32', 'shape': [10], 'description': 'Predictions per image category'}])

tf_model = mr.tensorflow.create_model(MODEL_NAME,
                                      metrics=metrics,
                                      input_example=input_example,
                                      model_schema=ModelSchema(input_schema=input_schema, output_schema=output_schema))

tf_model.save(export_path)   

1/1 [==============================] - 0s 140ms/step - loss: 1.0067 - accuracy: 0.5625
Saving trained model to: /srv/hops/staging/private_dirs/19b0fb823ca769217d995f45563455c4363cf74c556b80afe5762fe813fd9713/model-2959f4bb-a422-4525-b76d-3c975969678d
2022-07-09 17:52:11,313 INFO: Assets written to: /srv/hops/staging/private_dirs/19b0fb823ca769217d995f45563455c4363cf74c556b80afe5762fe813fd9713/model-2959f4bb-a422-4525-b76d-3c975969678d/assets
Done saving!
Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://hopsworks0.logicalclocks.com/p/119/models/mniste2e/1


Model(name: 'mniste2e', version: 1)

## Serve the MNIST classifier

#### Create a connection to Hopsworks

In [3]:
import hsml

# Connect with Hopsworks
conn = hsml.connection()

# Retrieve the model registry handle
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


### Query Model Repository for best Mnist model

In [4]:
# Get best version of the mnist model
best_model = mr.get_best_model(MODEL_NAME, "accuracy", "max")

print('Model name: ' + best_model.name)
print('Model version: ' + str(best_model.version))
print(best_model.training_metrics)

Model name: mniste2e
Model version: 1
{'accuracy': '0.5625'}


### Create a Deployment for the Trained Model

In [5]:
from hsml.inference_logger import InferenceLogger
from hsml.kafka_topic import KafkaTopic

kafka_topic = KafkaTopic()
my_logger = InferenceLogger(kafka_topic=kafka_topic, mode="ALL")

# Deploy the best version of the model
mnistclassifier = best_model.deploy(inference_logger=my_logger) # To deploy the model using KServe, set serving_tool to KSERVE


Setting number of replicas to default value '1'
Setting number of partitions to default value '1'
Setting up a Tensorflow Serving server to run the model
Deployment created, explore it at https://hopsworks0.logicalclocks.com/p/119/deployments/4
Before making predictions, start the deployment by using `.start()`


After the serving have been created, you can find it in the Hopsworks UI by going to the "Deployments" tab. You can also use the class attributes or the `.describe()` method of a deployment object to describe access its metadata.

In [6]:
print("Description of " + mnistclassifier.name)
mnistclassifier.describe()

Description of mniste2e
{
    "artifact_version": null,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2022-07-09T17:52:44Z",
    "creator": "Admin Admin",
    "id": 4,
    "inference_logging": "ALL",
    "kafka_topic_dto": {
        "name": "mniste2e-inf6486",
        "num_of_partitions": null,
        "num_of_replicas": null
    },
    "model_name": "mniste2e",
    "model_path": "/Projects/demo_ml_meb10000/Models/mniste2e",
    "model_server": "TENSORFLOW_SERVING",
    "model_version": 1,
    "name": "mniste2e",
    "predictor": null,
    "predictor_resources": {
        "limits": {
            "cores": 1,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 1,
            "gpus": 0,
            "memory": 1024
        }
    },
    "requested_instances": 1,
    "serving_tool": "DEFAULT"
}


## Classify Mnist samples using the Trained Model

### Start Deployment

In [7]:
if mnistclassifier.get_state().status == "Stopped":
    mnistclassifier.start()

  0%|          | 0/1 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [8]:
import time
while mnistclassifier.get_state().status != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(10)

### Send prediction requests to the Mnist model

In [9]:
for i in range(10):
    data = { "signature_name": "serving_default", "instances": [best_model.input_example] }
    predictions = mnistclassifier.predict(data)
    print(predictions)

{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]]}


## Monitor Prediction Logs

### Consume Prediction Requests and Responses using Kafka

All prediction requestst are automatically logged to Kafka which means that you can keep track for your model's performance and its predictions in a scalable manner.

In [10]:
from hops import kafka
from confluent_kafka import Producer, Consumer, KafkaError

Setup a Kafka consumer and subscribe to the topic containing the prediction logs

In [11]:
TOPIC_NAME = mnistclassifier.inference_logger.kafka_topic.name

config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config)
topics = [TOPIC_NAME]
consumer.subscribe(topics)

Read the Kafka Avro schema from Hopsworks and setup an Avro reader

In [12]:
json_schema = kafka.get_schema(TOPIC_NAME)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

Read messages from the Kafka topic, parse them with the Avro schema and print the results

In [13]:
import json

PRINT_INSTANCES=False
PRINT_PREDICTIONS=True

for i in range(0, 10):
    msg = consumer.poll(timeout=5.0)
    if msg is not None:
        value = msg.value()
        try:
            event_dict = kafka.parse_avro_msg(value, avro_schema)
            
            print("serving: {}, version: {}, timestamp: {},\n"\
                  "         http_response_code: {}, model_server: {}, serving_tool: {}".format(
                       event_dict["modelName"],
                       event_dict["modelVersion"],
                       event_dict["requestTimestamp"],
                       event_dict["responseHttpCode"],
                       event_dict["modelServer"],
                       event_dict["servingTool"]))
            
            if PRINT_INSTANCES:
                print("instances: {}\n".format(event_dict["inferenceRequest"]))
            if PRINT_PREDICTIONS:
                print("predictions: {}\n".format(json.loads(event_dict["inferenceResponse"])["predictions"][0]))
                      
        except Exception as e:
            print("A message was read but there was an error parsing it")
            print(e)
    else:
        print("timeout.. no more messages to read from topic")

serving: mniste2e, version: 1, timestamp: 1657389186893,
         http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

serving: mniste2e, version: 1, timestamp: 1657389187493,
         http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

serving: mniste2e, version: 1, timestamp: 1657389188033,
         http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

serving: mniste2e, version: 1, timestamp: 1657389188631,
         http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

serving: mniste2e, version: 1, timestamp: 1657389189191,
         http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predi